In [1]:
import instructor
import torch
import transformers
from langchain_community.embeddings import HuggingFaceInstructEmbeddings

print("All dependencies imported successfully!")

/Users/soehtetnaing/miniconda3/envs/new_chatbot_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/soehtetnaing/miniconda3/envs/new_chatbot_env/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


All dependencies imported successfully!


In [2]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
from langchain.document_loaders import PyPDFLoader

# Load the PDF file
pdf_path = "./about-nca.pdf" 
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# Display extracted text for verification
for doc in documents[:3]:  # Display first 3 extracted text parts
    print(doc.page_content)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)
Ignoring wrong pointing object 69 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)
Ignoring wrong pointing object 74 0 (offset 0)


Page 1 of 5  
Name: Nyein Chan Aung Position: Digital Product & Project Professional Address: Yangon, Myanmar Phone: 091212121  My name is Nyein Chan Aung. I was born in 7th Aug, 1990 and 35 years old.  I’m currently working as a Product Manager at Village Link, one of the subsidiaries of Myanma Awba. I am mainly responsible for product management, UI/UX research, user analysis, QA testing, and the product launch of our company’s many products. I am leading the development of a delivery social agricultural platform with registered farmers. My ability to understand the system and database design allowed me to migrate the hundred thousand user-generated datasets from the legacy system to a completely new platform in 2021.  I started my career in the tech industry as a developer while I was studying Business Information Technology. After that, I moved forward into product development and management when I found my enthusiasm for the art of product development lifecycle applying Agile Meth

In [4]:
len(documents)

5

In [5]:
documents

[Document(page_content='Page 1 of 5  \nName: Nyein Chan Aung Position: Digital Product & Project Professional Address: Yangon, Myanmar Phone: 091212121  My name is Nyein Chan Aung. I was born in 7th Aug, 1990 and 35 years old.  I’m currently working as a Product Manager at Village Link, one of the subsidiaries of Myanma Awba. I am mainly responsible for product management, UI/UX research, user analysis, QA testing, and the product launch of our company’s many products. I am leading the development of a delivery social agricultural platform with registered farmers. My ability to understand the system and database design allowed me to migrate the hundred thousand user-generated datasets from the legacy system to a completely new platform in 2021.  I started my career in the tech industry as a developer while I was studying Business Information Technology. After that, I moved forward into product development and management when I found my enthusiasm for the art of product development life

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 100,  # Chunk size of 50 characters
    chunk_overlap= 15  # 10-character overlap
)

chunks = text_splitter.split_documents(documents)
print(f"Total Chunks: {len(chunks)}")


Total Chunks: 116


In [26]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings
import os

# Initialize the embedding model
embedding_model = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-base"
)

# Define a directory and file to save the model
save_dir = "./embedding_model"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "instructor_model.pth")

# Save the underlying INSTRUCTOR model (accessed via .client)
torch.save(embedding_model.client.state_dict(), save_path)
print(f"Embedding model saved to {save_path}")

load INSTRUCTOR_Transformer
max_seq_length  512
Embedding model saved to ./embedding_model/instructor_model.pth


In [ ]:
#save vector locally
from langchain.vectorstores import FAISS


vector_path = 'vector-store'

vectordb = FAISS.from_documents(
    documents = chunks,
    embedding = embedding_model
)

db_file_name = 'nyein'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nca' #default index
)

In [ ]:
#calling vector from local
vector_path = 'vector-store'
db_file_name = 'nyein'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path=os.path.join(vector_path, db_file_name),
    embeddings=embedding_model,
    index_name='nca',
    allow_dangerous_deserialization=True  # ✅ Enable safe deserialization
)

print("✅ FAISS index loaded successfully!")

✅ FAISS index loaded successfully!


In [11]:
#ready to use
retriever = vectordb.as_retriever()

In [12]:

retriever.get_relevant_documents("education?")

/Users/soehtetnaing/miniconda3/envs/new_chatbot_env/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='university students.', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 4}),
 Document(page_content='Laws (LL. B) • Yangon University of Distance Education, Yangon, Myanmar (2007 - 2010)  WORK', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 1}),
 Document(page_content='technical industries (education, finance, agriculture). My major objective is to contribute to the', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 1}),
 Document(page_content='provide value to clients and users.    EDUCATION Master of Data Science and Artificial Intelligence', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 1})]

In [13]:

retriever.get_relevant_documents("language?")

[Document(page_content='to scan the applicant’s National Identity.  Web Application : https://maha-ekyc.htwettoe.com  OTHER', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 3}),
 Document(page_content='and junior developers & designers on technical, design, and UI/UX. • Responsible for communicating', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 2}),
 Document(page_content='provide value to clients and users.    EDUCATION Master of Data Science and Artificial Intelligence', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 1}),
 Document(page_content='and junior developers with technical guidance. Developer Global Connect Asia  June 2013 – September', metadata={'source': '/Users/soehtetnaing/Documents/GitHub/NLP_A6/about-nca.pdf', 'page': 2})]

In [14]:
from langchain import PromptTemplate

prompt_template = """
Please answer the following question accurately based on the provided context of a person named Nyein Chan Aung.
Context:
{context}

Question: {question}

Gentle & Informative Answer:
""".strip()

# Load Prompt Template
PROMPT = PromptTemplate.from_template(prompt_template)


In [15]:
questions = [
    "How old are you?",
    "What is your highest level of education?",
    "What major or field of study did you pursue during your education?",
    "How many years of work experience do you have?",
    "What type of work or industry have you been involved in?",
    "Can you describe your current role or job responsibilities?",
    "What are your core beliefs regarding the role of technology in shaping society?",
    "How do you think cultural values should influence technological advancements?",
    "As a master’s student, what is the most challenging aspect of your studies so far?",
    "What specific research interests or academic goals do you hope to achieve during your time as a master’s student?"
]

In [ ]:
# gsk_1q QSWoZQVvo hKVMpJwFU WGdyb3FY9tQ HBpYvOzgly Pe7DtSrrrJy

In [ ]:
# Set Groq API Key (replace with your key)
os.environ["GROQ_API_KEY"] = "your_groq_api_key"


In [17]:
from groq import Groq

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
models = client.models.list()

for model in models:
    print(model)


('data', [Model(id='deepseek-r1-distill-qwen-32b', created=1738891590, object='model', owned_by='DeepSeek / Alibaba Cloud', active=True, context_window=131072, public_apps=None), Model(id='allam-2-7b', created=1737672203, object='model', owned_by='SDAIA', active=True, context_window=4096, public_apps=None), Model(id='llama-3.2-1b-preview', created=1727224268, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='llama-3.2-3b-preview', created=1727224290, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='llama-guard-3-8b', created=1693721698, object='model', owned_by='Meta', active=True, context_window=8192, public_apps=None), Model(id='llama-3.3-70b-versatile', created=1733447754, object='model', owned_by='Meta', active=True, context_window=32768, public_apps=None), Model(id='mistral-saba-24b', created=1739996492, object='model', owned_by='Mistral AI', active=True, context_window=32768, public_apps=

In [18]:
from langchain_groq import ChatGroq
import os


# Load Groq Model
groq_model = ChatGroq(model_name="llama-3.2-3b-preview", temperature=0.7)

def groq_response(question):
    input_document = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in input_document])
    
    prompt = PROMPT.format(context=context, question=question)
    return groq_model.invoke(prompt)



In [19]:
# Test the model
print(groq_response("Where did Nyein Chan Aung study?"))

content='Nyein Chan Aung studied at the Yangon University of Distance Education in Yangon, Myanmar.' response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 172, 'total_tokens': 194, 'completion_time': 0.014154417, 'prompt_time': 0.029238193, 'queue_time': 0.019885576, 'total_time': 0.04339261}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None} id='run-6cfe4af9-ac8b-4b6c-8e7e-ab49712cb2ac-0'


In [20]:
# Get responses for each question
answers = []
for question in questions:
    response = groq_response(question)
    answers.append(response)
    print(f"Bot Answer for '{question}':", response)

Bot Answer for 'How old are you?': content='You mentioned earlier that you are 35 years old.' response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 189, 'total_tokens': 201, 'completion_time': 0.007719526, 'prompt_time': 0.032260426, 'queue_time': 0.016009311, 'total_time': 0.039979952}, 'model_name': 'llama-3.2-3b-preview', 'system_fingerprint': 'fp_a926bfdce1', 'finish_reason': 'stop', 'logprobs': None} id='run-41344d90-ef3b-472e-9dec-c14cffd2907a-0'
Bot Answer for 'What is your highest level of education?': content="Nyein Chan Aung's highest level of education is a Master's degree in Data Science and Artificial Intelligence, however, the exact university is not specified in the provided context." response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 147, 'total_tokens': 184, 'completion_time': 0.023835819, 'prompt_time': 0.025218452, 'queue_time': 0.019834397, 'total_time': 0.049054271}, 'model_name': 'llama-3.2-3b-preview', 'system_fing

In [21]:
from langchain.chains import LLMChain  # Import LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT

# Question Generator (Rewrites User Queries)
question_generator = LLMChain(
    llm=groq_model,
    prompt=CONDENSE_QUESTION_PROMPT,
    verbose=True
)

# Retrieval Chain (Fetches Relevant Documents)
doc_chain = load_qa_chain(
    llm=groq_model,
    chain_type="stuff",
    prompt=PROMPT,
    verbose=True
)

# Conversational Memory
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Conversational Retrieval Chain
groq_chain = ConversationalRetrievalChain(
    retriever=retriever,  # FAISS Retriever
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True
)

In [ ]:
# Generate responses
answers = []
for question in questions:
    response = groq_chain.invoke({"question": question})
    answers.append(response)
    print(f"Bot Answer for '{question}':", response)



> Entering new ConversationalRetrievalChain chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Please answer the following question accurately based on the provided context of a person named Nyein Chan Aung.
Context:
Myanmar Phone: 091212121  My name is Nyein Chan Aung. I was born in 7th Aug, 1990 and 35 years old.

35 years old.  I’m currently working as a Product Manager at Village Link, one of the subsidiaries

Oct 2016 - Jun 2017   	COMMUNITY INVOLVEMENT  • Youth Entrepreneurship Program (2022 - Mentor) o

a completely new platform in 2021.  I started my career in the tech industry as a developer while I

Question: How old are you?

Gentle & Informative Answer:

> Finished chain.

> Finished chain.

> Finished chain.
Bot Answer for 'How old are you?': {'question': 'How old are you?', 'chat_history': [], 'answer': 'Nyein Chan Aung, you are 35 years old.', 'source_documents': [Document(page_content='Myanmar Phone: 0912